# Variable Group Face Network (VarGFaceNet) with Adaptive Margin Loss AdaFace

## Clone VarGFaceNet Repo

In [ ]:
!git clone https://github.com/daffatm/VarGFaceNet.git

Cloning into 'VarGFaceNet'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 262 (delta 124), reused 209 (delta 71), pack-reused 0
Receiving objects: 100% (262/262), 33.01 MiB | 32.47 MiB/s, done.
Resolving deltas: 100% (124/124), done.


## Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=1)

Mounted at /content/drive


## Prepare all dataset

In [ ]:
!tar -xf "/content/drive/MyDrive/Face Recognition/FR Dataset/casia/casia-112x112.tar" -C "/content"
print("Untar CASIA Done")

Untar CASIA Done


In [ ]:
!unzip -qq "/content/drive/MyDrive/Face Recognition/FR Dataset/lfw/lfw_sr.zip" -d "/content"
print("Unzip LFW Done")

Unzip LFW Done


In [ ]:
!unzip -qq "/content/drive/MyDrive/Face Recognition/FR Dataset/tinyface/tinyface_aligned_sr.zip" -d "/content"
print("Unzip TinyFace Done")

Unzip TinyFace Done


### Create LFW Lowres

In [ ]:
# Tidak perlu di run karna data pada zip sudah di resize
# Create Lowres LFW
!python "/content/VarGFaceNet/preprocess_lfw.py"

Resizing to lfw-56x56
Progress Done: 1/1
Resize to lfw-56x56 Done!
Error Data: 0
[]
Resizing to lfw-28x28
Progress Done: 1/1
Resize to lfw-28x28 Done!
Error Data: 0
[]
Resizing to lfw-14x14
Progress Done: 1/1
Resize to lfw-14x14 Done!
Error Data: 0
[]


### Upscalling with Real-ESRGAN

Cell ini tidak perlu di run, karna zip sudah berisi dataset yang sudah di upscaling denga Real-ESRGAN

In [ ]:
# # Clone Real-ESRGAN and enter the Real-ESRGAN
# !git clone https://github.com/xinntao/Real-ESRGAN.git
# %cd Real-ESRGAN
# # Set up the environment
# !pip install basicsr
# !pip install facexlib
# !pip install gfpgan
# !pip install -r requirements.txt
# !python setup.py develop

In [ ]:
# import os
# import warnings
# import subprocess
# import sys
# import cv2
# import glob
# import multiprocessing

# from basicsr.archs.rrdbnet_arch import RRDBNet
# from basicsr.utils.download_util import load_file_from_url
# from realesrgan import RealESRGANer
# from realesrgan.archs.srvgg_arch import SRVGGNetCompact
# from gfpgan import GFPGANer

# warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [ ]:
# # LFW
# if not os.path.exists("/content/lfw/SR/lfw-56x56"):
#     os.makedirs("/content/lfw/SR/lfw-56x56")
# if not os.path.exists("/content/lfw/SR/lfw-28x28"):
#     os.makedirs("/content/lfw/SR/lfw-28x28")
# if not os.path.exists("/content/lfw/SR/lfw-14x14"):
#     os.makedirs("/content/lfw/SR/lfw-14x14")

# # TinyFace
# if not os.path.exists("/content/tinyface/SR/Gallery_Distractor"):
#     os.makedirs("/content/tinyface/SR/Gallery_Distractor")
# if not os.path.exists("/content/tinyface/SR/Gallery_Match"):
#     os.makedirs("/content/tinyface/SR/Gallery_Match")
# if not os.path.exists("/content/tinyface/SR/Probe"):
#     os.makedirs("/content/tinyface/SR/Probe")

In [ ]:
# model_name = "realesr-general-x4v3" # S size model, more light
# model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64,
#                         num_conv=32, upscale=4, act_type='prelu')
# netscale = 4
# file_url = [
#     'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-wdn-x4v3.pth',
#     'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth'
# ]

# # Get pretrained model
# model_path = os.path.join('weights', model_name + '.pth')
# if not os.path.isfile(model_path):
#     ROOT_DIR = os.path.dirname(os.path.abspath("Real-ESRGAN"))
#     for url in file_url:
#         # model_path will be updated
#         model_path = load_file_from_url(
#             url=url, model_dir=os.path.join(ROOT_DIR, 'weights'), progress=True, file_name=None)

# def face_enhance(outscale, upsampler):
#     face_enhancer = GFPGANer(
#         model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
#         upscale=outscale,
#         arch='clean',
#         channel_multiplier=2,
#         bg_upsampler=upsampler)

#     return face_enhancer

# def count_files(root_folder):
#     count = 0
#     for root, dirs, files in os.walk(root_folder):
#         count += len(files)
#     return count

# def make_sr(input_folder, output_folder, outscale, dn=0.5, model_path=model_path):
#     # use dni to control the denoise strength
#     dni_weight = None
#     if dn != 1:
#         wdn_model_path = model_path.replace('realesr-general-x4v3', 'realesr-general-wdn-x4v3')
#         model_path = [model_path, wdn_model_path]
#         dni_weight = [dn, 1 - dn]

#     # restorer
#     upsampler = RealESRGANer(
#         scale=netscale,
#         model_path=model_path,
#         dni_weight=dni_weight,
#         model=model,
#         tile=0,
#         tile_pad=10,
#         pre_pad=0,
#         half=True,
#         gpu_id=0)

#     # Face restoration
#     face_enhancer = face_enhance(outscale, upsampler)

#     os.makedirs(output_folder, exist_ok=True)

#     total_files = count_files(input_folder)
#     total_counts = 0

#     print("Start Process...")
#     for root, _, files in os.walk(input_folder):
#         for idx, file in enumerate(files):
#             path = os.path.join(root, file)
#             imgname, extension = os.path.splitext(file)
#             # print('Testing', idx, imgname)
#             total_counts += 1
#             sys.stdout.write("\rProgress SR: {0}/{1}".format(total_counts, total_files))
#             sys.stdout.flush()

#             img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
#             if len(img.shape) == 3 and img.shape[2] == 4:
#                 img_mode = 'RGBA'
#             else:
#                 img_mode = None

#             try:
#                 _, _, out = face_enhancer.enhance(img, has_aligned=False, only_center_face=False,
#                                                   paste_back=True)
#             except RuntimeError as error:
#                 print('Error', error)
#                 print('If you encounter CUDA out of memory, try to set --tile with a smaller number.')
#             else:
#                 extension = extension[1:]
#                 if img_mode == 'RGBA':  # RGBA images should be saved in png format
#                     extension = 'png'
#                 output_path = os.path.join(output_folder, os.path.relpath(root, input_folder),
#                                            f'{imgname}.{extension}')
#                 os.makedirs(os.path.dirname(output_path), exist_ok=True)
#                 cv2.imwrite(output_path, out)
#     print("SR Done!")

Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-wdn-x4v3.pth" to /content/Real-ESRGAN/weights/realesr-general-wdn-x4v3.pth



100%|██████████| 4.66M/4.66M [00:00<00:00, 29.2MB/s]


Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth" to /content/Real-ESRGAN/weights/realesr-general-x4v3.pth



100%|██████████| 4.66M/4.66M [00:00<00:00, 30.1MB/s]


In [ ]:
# # LFW
# make_sr("/content/lfw/lfw-56x56", "/content/lfw/SR/lfw-56x56", 1)
# make_sr("/content/lfw/lfw-28x28", "/content/lfw/SR/lfw-28x28", 1)
# make_sr("/content/lfw/lfw-14x14", "/content/lfw/SR/lfw-14x14", 1)

Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/Real-ESRGAN/gfpgan/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 114MB/s] 


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/Real-ESRGAN/gfpgan/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 110MB/s]


Downloading: "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth" to /usr/local/lib/python3.10/dist-packages/gfpgan/weights/GFPGANv1.3.pth



100%|██████████| 332M/332M [00:04<00:00, 83.4MB/s]


Start Process...
Progress SR: 13233/13233SR Done!
Start Process...
Progress SR: 13233/13233SR Done!
Start Process...
Progress SR: 13233/13233SR Done!


In [ ]:
# %cd ..
# !ls

In [ ]:
# !zip -r "/content/lfw_sr.zip" "/content/lfw"

In [ ]:
# !cp /content/lfw_sr.zip "/content/drive/MyDrive/Face Recognition/FR Dataset/lfw"

In [ ]:
# %cd Real-ESRGAN
# !ls

/content/Real-ESRGAN
assets			 inference_realesrgan_video.py	realesrgan.egg-info
CODE_OF_CONDUCT.md	 inputs				requirements.txt
cog_predict.py		 LICENSE			scripts
cog.yaml		 MANIFEST.in			setup.cfg
docs			 options			setup.py
experiments		 README_CN.md			tests
gfpgan			 README.md			VERSION
inference_realesrgan.py  realesrgan			weights


In [ ]:
# # TinyFace
# make_sr("/content/tinyface/Gallery_Distractor", "/content/tinyface/SR/Gallery_Distractor", 1)
# make_sr("/content/tinyface/Gallery_Match", "/content/tinyface/SR/Gallery_Match", 1)
# make_sr("/content/tinyface/Probe", "/content/tinyface/SR/Probe", 1)

Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/Real-ESRGAN/gfpgan/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 321MB/s] 


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/Real-ESRGAN/gfpgan/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 313MB/s]


Downloading: "https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth" to /usr/local/lib/python3.10/dist-packages/gfpgan/weights/GFPGANv1.3.pth



100%|██████████| 332M/332M [00:02<00:00, 154MB/s]


Start Process...
Progress SR: 153428/153428SR Done!
Start Process...
Progress SR: 4443/4443SR Done!
Start Process...
Progress SR: 3728/3728SR Done!


In [ ]:
# %cd ..
# !ls

/content
drive  Real-ESRGAN  sample_data  tinyface


In [ ]:
# !zip -r "/content/tinyface_aligned_sr.zip" "/content/tinyface"

In [ ]:
# !cp /content/tinyface_aligned_sr.zip "/content/drive/MyDrive/Face Recognition/FR Dataset/tinyface"

In [ ]:
# from google.colab import files
# files.download('tinyface_aligned_sr.zip')

## Change dir to VarGFaceNet

In [ ]:
%cd "/content/VarGFaceNet"

/content/VarGFaceNet


## Train model

In [ ]:
# Edit config.py first before run !!!
# ========================================================================
# Casia Path = '/content/casia-112x112'
# LFW Path = '/content/lfw'
# TinyFace Path = '/content/tinyface'
# Save path = '../drive/MyDrive/Face Dataset/vargfacenet/model'
# Resume path = '../drive/MyDrive/Face Dataset/vargfacenet/model/....ckpt'
# ========================================================================

# !python train.py

defining casia dataloader...
casia with augmentation...
defining lfw dataloader...
defining vargfacenet model...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 40, 112, 112]           1,080
       BatchNorm2d-2         [-1, 40, 112, 112]              80
             PReLU-3         [-1, 40, 112, 112]              40
            Conv2d-4           [-1, 40, 56, 56]           2,880
       BatchNorm2d-5           [-1, 40, 56, 56]              80
             PReLU-6           [-1, 40, 56, 56]              40
            Conv2d-7           [-1, 40, 56, 56]           1,600
       BatchNorm2d-8           [-1, 40, 56, 56]              80
  Separable_Conv2d-9           [-1, 40, 56, 56]               0
           Conv2d-10           [-1, 40, 56, 56]           2,880
      BatchNorm2d-11           [-1, 40, 56, 56]              80
            PReLU-12           [-1, 40, 56, 56]       

In [ ]:
# from google.colab import runtime
# runtime.unassign()

## Model Benchmark

In [ ]:
# !pip install torchinfo
# !pip install -U 'git+https://github.com/facebookresearch/fvcore'
!pip install deepspeed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.8/808.8 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 17.7 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.9.4-py3-none-any.whl size=844002 sha256=8b342c0fb00f643217a872686ac74edfe34d82044719e0105132b4e7c62c2252
  Stored in directory: /root/.cache/pip/wheels/2d/ae/38/1d1c49ac8687c5808b3732e3541b6c896459fb8404763eb98b
Successfully built deepspeed


In [ ]:
import torch
import numpy as np
from core import model, shuffleface
# from torchinfo import summary
# from fvcore.nn import FlopCountAnalysis , flop_count_table
from deepspeed.profiling.flops_profiler import get_model_profile
from deepspeed.accelerator import get_accelerator
from torch.profiler import profile, record_function, ProfilerActivity

[2023-06-17 07:49:41,624] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
def benchmark():
    # net = model.VarGFaceNet()
    net = shuffleface.ShuffleFaceNet()
    net = net.eval()
    with get_accelerator().device(0):
        flops, macs, params = get_model_profile(model=net, # model
                                        input_shape=(1, 3, 112, 112), # input shape to the model. If specified, the model takes a tensor with this shape as the only positional argument.
                                        args=None, # list of positional arguments to the model.
                                        kwargs=None, # dictionary of keyword arguments to the model.
                                        print_profile=True, # prints the model graph with the measured profile attached to each module
                                        detailed=False, # print the detailed profile
                                        module_depth=-1, # depth into the nested modules, with -1 being the inner most modules
                                        top_modules=1, # the number of top modules to print aggregated profile
                                        warm_up=30, # the number of warm-ups before measuring the time of each module
                                        as_string=True, # print raw numbers (e.g. 1000) or as human-readable strings (e.g. 1k)
                                        output_file="/content/shuffleface profile.txt", # path to the output file. If None, the profiler prints to stdout.
                                        ignore_modules=None) # the list of modules to ignore in the profiling
    print(f"FLOPs: {flops}")
    print(f"MACs: {macs}")
    print(f"Params: {params}")

def time_benchmark(gpu=False):
    device = "cuda" if gpu==True else "cpu"
    # net = model.VarGFaceNet().to(device)
    net = shuffleface.ShuffleFaceNet().to(device)
    net.eval()

    start, end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    reps = 300
    timings = np.zeros((reps, 1))
    input = torch.randn(1, 3, 112, 112).to(device)

    # GPU warmup
    for _ in range(10):
        embedding, norm = net(input)

    # Measure performance
    with torch.no_grad():
        for rep in range(reps):
            start.record()
            embedding, norm = net(input)
            end.record()
            # Wait for GPU to sync
            torch.cuda.synchronize()
            curr_time = start.elapsed_time(end)
            timings[rep] = curr_time

    mean_syn = np.sum(timings) * 10 ** -3 / reps
    print(f"{device} time total: {round(mean_syn*1000, 2)}ms")

    # embedding, _ = net(input)

    # with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    #     with record_function("model_inference"):
    #         embedding, _ = net(input)

    # print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

In [ ]:
# Model Profiling
benchmark()

FLOPs: 584.59 M
MACs: 288.82 MMACs
Params: 2.66 M


In [ ]:
# Model benchmark without GPU
time_benchmark(gpu=False)

cpu time total: 22.36ms


In [ ]:
# Model benchmark with GPU
time_benchmark(gpu=True)

cuda time total: 13.05ms


## LFW Evaluation

In [ ]:
from lfw_eval import getFeatureFromTorch, evaluation_10_fold
from inference import inference, face_verification
from config import LFW_DATA_DIR
import numpy as np
import os

### LFW 112x112

In [ ]:
model = "/content/drive/MyDrive/Face Recognition/model/VarGFaceNet/AdaFace/CASIA_VarGFaceNet_20230605_042815/022.ckpt"
save_dir = "result/lfw-112x112_result.mat"
imgl = "/content/lfw/lfw-112x112/Adel_Al-Jubeir/Adel_Al-Jubeir_0001.jpg"
imgr = "/content/lfw/lfw-112x112/Adel_Al-Jubeir/Adel_Al-Jubeir_0002.jpg"
# imgr = "/content/lfw/lfw-112x112/Abdoulaye_Wade/Abdoulaye_Wade_0002.jpg" # different face

In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "lfw-112x112", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


extracing deep features from the face pair 6000/6000
-------------------------------------
1    99.00
2    99.00
3    99.17
4    98.33
5    98.67
6    99.50
7    98.67
8    99.33
9    99.67
10    99.50
--------
AVE ACC    99.08
AVE THS    0.23


In [ ]:
# Face Verification test
face_verification(imgl, imgr, resume=model, gpu=True)

Jarak Embedding: 0.7763283252716064
Gambar tersebut adalah gambar dari orang yang sama


### LFW 56x56

In [ ]:
model = "/content/drive/MyDrive/Face Recognition/model/VarGFaceNet/AdaFace/CASIA_VarGFaceNet_20230605_042815/022.ckpt"
save_dir = "result/lfw-56x56_result.mat"
imgl = "/content/lfw/SR/lfw-56x56/Adel_Al-Jubeir/Adel_Al-Jubeir_0001.jpg"
imgr = "/content/lfw/SR/lfw-56x56/Adel_Al-Jubeir/Adel_Al-Jubeir_0002.jpg"
# imgr = "/content/lfw/SR/lfw-56x56/Abdoulaye_Wade/Abdoulaye_Wade_0002.jpg" # different face

In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "lfw-56x56", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

extracing deep features from the face pair 6000/6000
-------------------------------------
1    98.83
2    98.67
3    98.67
4    98.00
5    98.67
6    99.33
7    98.83
8    99.00
9    99.83
10    99.33
--------
AVE ACC    98.92
AVE THS    0.23


In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "SR/lfw-56x56", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

extracing deep features from the face pair 6000/6000
-------------------------------------
1    98.67
2    98.67
3    99.00
4    97.00
5    98.17
6    99.33
7    98.50
8    98.50
9    99.33
10    99.00
--------
AVE ACC    98.62
AVE THS    0.21


In [ ]:
# Face Verification test
face_verification(imgl, imgr, resume=model, gpu=True)

Jarak Embedding: 0.7426758408546448
Gambar tersebut adalah gambar dari orang yang sama


### LFW 28x28

In [ ]:
model = "/content/drive/MyDrive/Face Recognition/model/VarGFaceNet/AdaFace/CASIA_VarGFaceNet_20230605_042815/022.ckpt"
save_dir = "result/lfw-28x28_result.mat"
imgl = "/content/lfw/SR/lfw-28x28/Adel_Al-Jubeir/Adel_Al-Jubeir_0001.jpg"
imgr = "/content/lfw/SR/lfw-28x28/Adel_Al-Jubeir/Adel_Al-Jubeir_0002.jpg"
# imgr = "/content/lfw/SR/lfw-28x28/Abdoulaye_Wade/Abdoulaye_Wade_0002.jpg" # different face

In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "lfw-28x28", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

extracing deep features from the face pair 6000/6000
-------------------------------------
1    95.83
2    96.50
3    96.00
4    95.00
5    95.67
6    97.17
7    97.83
8    96.33
9    97.33
10    96.17
--------
AVE ACC    96.38
AVE THS    0.17


In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "SR/lfw-28x28", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

extracing deep features from the face pair 6000/6000
-------------------------------------
1    95.67
2    95.33
3    96.50
4    94.33
5    94.17
6    95.33
7    97.33
8    95.67
9    96.67
10    95.00
--------
AVE ACC    95.60
AVE THS    0.17


In [ ]:
# Face Verification test
face_verification(imgl, imgr, resume=model, gpu=True)

Jarak Embedding: 0.568160355091095
Gambar tersebut adalah gambar dari orang yang berbeda


### LFW 14x14

In [ ]:
model = "/content/drive/MyDrive/Face Recognition/model/VarGFaceNet/AdaFace/CASIA_VarGFaceNet_20230605_042815/022.ckpt"
save_dir = "result/lfw-14x14_result.mat"
imgl = "/content/lfw/SR/lfw-14x14/Adel_Al-Jubeir/Adel_Al-Jubeir_0001.jpg"
imgr = "/content/lfw/SR/lfw-14x14/Adel_Al-Jubeir/Adel_Al-Jubeir_0002.jpg"
# imgr = "/content/lfw/SR/lfw-14x14/Abdoulaye_Wade/Abdoulaye_Wade_0002.jpg" # different face

In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "lfw-14x14", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

extracing deep features from the face pair 6000/6000
-------------------------------------
1    73.00
2    69.83
3    70.83
4    72.50
5    69.83
6    72.67
7    69.83
8    71.67
9    70.00
10    72.67
--------
AVE ACC    71.28
AVE THS    0.18


In [ ]:
getFeatureFromTorch(LFW_DATA_DIR, "SR/lfw-14x14", save_dir, resume=model, gpu=True)
ACCs, thresholds = evaluation_10_fold(save_dir)

print("\n-------------------------------------")
for i in range(len(ACCs)):
    print('{}    {:.2f}'.format(i+1, ACCs[i] * 100))
print('--------')
print('AVE ACC    {:.2f}'.format(np.mean(ACCs) * 100))
print('AVE THS    {:.2f}'.format(np.mean(thresholds)))

extracing deep features from the face pair 6000/6000
-------------------------------------
1    80.83
2    78.83
3    80.33
4    80.67
5    77.67
6    81.50
7    81.83
8    77.67
9    79.33
10    80.00
--------
AVE ACC    79.87
AVE THS    0.10


In [ ]:
# Face Verification test
face_verification(imgl, imgr, resume=model, gpu=True)

Jarak Embedding: 0.5059691667556763
Gambar tersebut adalah gambar dari orang yang berbeda


## TinyFace Evaluation

In [ ]:
from tinyface_eval import load_model, prepare_dataloader, infer, calc_accuracy
import tinyface_helper
from config import TINYFACE_DATA_DIR
import pandas as pd
import os

In [ ]:
resume = "/content/drive/MyDrive/Face Recognition/vargfacenet/model/AdaFace/CASIA_VarGFaceNet_20230603_061119/022.ckpt"
save_dir = "result"

In [ ]:
# load model
model = load_model(resume)
# tinyface_test = tinyface_helper.TinyFaceTest(tinyface_root=os.path.join(TINYFACE_DATA_DIR, "SR"))
tinyface_test = tinyface_helper.TinyFaceTest(tinyface_root=TINYFACE_DATA_DIR)

probe_loader = prepare_dataloader(tinyface_test.probe_paths, 32, num_workers=5)
gallery_loader = prepare_dataloader(tinyface_test.gallery_paths, 32, num_workers=5)

print('probe images: {}'.format(len(tinyface_test.probe_paths)))
print('gallery images: {}'.format(len(tinyface_test.gallery_paths)))

probe_features = infer(model, probe_loader, use_flip_test=True)
gallery_features = infer(model, gallery_loader, use_flip_test=True)
calc_accuracy(tinyface_test, probe_features, gallery_features, save_dir, do_norm=True)
print("\n")
df_result = pd.read_csv('result/tinyface_result.csv')
df_result

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


probe images: 3728
gallery images: 157871


100%|██████████| 4934/4934 [07:24<00:00, 11.10it/s]


[46.08369098712446, 52.57510729613733, 55.391630901287556, 58.15450643776824]




,rank,values
0,1,46.083691
1,5,52.575107
2,10,55.391631
3,20,58.154506


In [ ]:
# load model
model = load_model(resume)
tinyface_test = tinyface_helper.TinyFaceTest(tinyface_root=os.path.join(TINYFACE_DATA_DIR, "SR"))
# tinyface_test = tinyface_helper.TinyFaceTest(tinyface_root=TINYFACE_DATA_DIR)

probe_loader = prepare_dataloader(tinyface_test.probe_paths, 32, num_workers=5)
gallery_loader = prepare_dataloader(tinyface_test.gallery_paths, 32, num_workers=5)

print('probe images: {}'.format(len(tinyface_test.probe_paths)))
print('gallery images: {}'.format(len(tinyface_test.gallery_paths)))

probe_features = infer(model, probe_loader, use_flip_test=True)
gallery_features = infer(model, gallery_loader, use_flip_test=True)
calc_accuracy(tinyface_test, probe_features, gallery_features, save_dir, do_norm=True)
print("\n")
df_result = pd.read_csv('result/tinyface_result.csv')
df_result

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


probe images: 3728
gallery images: 157871


100%|██████████| 4934/4934 [07:24<00:00, 11.09it/s]


[45.037553648068666, 51.636266094420606, 54.613733905579394, 57.64484978540773]




,rank,values
0,1,45.037554
1,5,51.636266
2,10,54.613734
3,20,57.644850


### Experimental

In [ ]:
# Ga bisa di run, makan RAM gede bngt, bakal auto crash
model = load_model(resume)
tinyface_test = tinyface_helper.TinyFaceTest(tinyface_root=os.path.join(TINYFACE_DATA_DIR, "SR"))
print('total images : {}'.format(len(tinyface_test.image_paths)))

dataloader = prepare_dataloader(tinyface_test.image_paths, 16, num_workers=2)
features = infer(model, dataloader, use_flip_test=True)
results = tinyface_test.test_identification(features, ranks=[1,5,20])
print(results)
pd.DataFrame({'rank':[1,5,20], 'values':results}).to_csv(os.path.join(save_dir, 'result.csv'))

total images : 169403


100%|██████████| 10588/10588 [14:27<00:00, 12.20it/s]


mate probes: 3728, non mate probes: 0
